In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
housing_data = '/Users/masoninman/Dropbox/housing/'

In [ ]:
# PVWatts parameters (determines file that is read)
tilt = 30
azimuth = 210

# PV watts

In [ ]:
def read_pvwatts(tilt, azimuth):
    # Read NREL generation data (PVWatts) for particular set-up
    # For Caswell house, set up with highest monthly average production across the year is:
    # tilt 30, azimuth 210

    # The data is hourly

    pvwatts_file = f'pvwatts_hourly 10 kW az {azimuth} tilt {tilt}.csv'

    pvwatts_header = pd.read_csv(
        housing_data + 'NREL PVWatts generation/' + 
        pvwatts_file,
        header = 0,
        nrows = 13,
        names = ['category', 'value'],
    )
    pvwatts_header = pvwatts_header.set_index('category')['value']
    
    # TEST: check that tilt & azimuth are as expected
    if int(pvwatts_header.at['Array Tilt (deg):']) != tilt:
        print("Error!")
    if int(pvwatts_header.at['Array Azimuth (deg):']) != azimuth:
        print("Error!")
    # END OF TEST

    pvwatts = pd.read_csv(
        housing_data + 'NREL PVWatts generation/' + 
        pvwatts_file,
        header = 14,
    )

    # remove totals
    pvwatts = pvwatts[pvwatts['Month']!='Totals']
    
    # set dtypes
    for col in ['Month', 'Day', 'Hour']:
        pvwatts[col] = pvwatts[col].astype(int)
        
    # rename columns
    pvwatts = pvwatts.rename(columns={
        'DC Array Output (W)': 'DC Array Output (Wh)',
        'AC System Output (W)': 'AC System Output (Wh)',
    })
    
    return pvwatts, pvwatts_header

In [ ]:
def pvwatts_set_datetime_index(pvwatts):
    datetime = '2018-' 
    datetime += pvwatts['Month'].astype(str).str.zfill(2) + '-'
    datetime += pvwatts['Day'].astype(str).str.zfill(2) + ' '
    datetime += pvwatts['Hour'].astype(str).str.zfill(2) + ':00:00'
    datetime = pd.to_datetime(datetime)
    pvwatts['Datetime'] = datetime
    pvwatts = pvwatts.set_index('Datetime')
    
    return pvwatts

In [ ]:
def scale_pvwatts(pvwatts, pv_kwh):
    # default size is 10 kWh
    scaling_factor = pv_kwh / 10
    # print(f"scaling_factor: {scaling_factor}")
    
    for col in ['DC Array Output (Wh)', 'AC System Output (Wh)']:
        pvwatts[col] = pvwatts[col] * scaling_factor
    
    return pvwatts

In [ ]:
pvwatts, pvwatts_header = read_pvwatts(tilt, azimuth)
pvwatts = pvwatts_set_datetime_index(pvwatts)

# ResStock

In [ ]:
# # calculate inverter loss
# # (this this simply assumed in the model? the parameters at the top say 'invert efficiency' = 96
# totals_index = pvwatts[pvwatts['Month']=='Totals'].index[0]
# dc = pvwatts.at[totals_index, 'DC Array Output (W)']
# ac = pvwatts.at[totals_index, 'AC System Output (W)']
# inverter_loss = (dc - ac) / dc
# inverter_loss

In [ ]:
def resstock_read():
    # Read ResStock data for Travis County
    # Note: should the code have an extra zero? g48004530
    resstock_path = housing_data + 'NREL ResStock/'
    resstock_file = 'g4804530-single-family_detached.csv'
    resstock = pd.read_csv(resstock_path + resstock_file)

    resstock['timestamp'] = pd.to_datetime(resstock['timestamp'])
    resstock = resstock.set_index('timestamp')
    
    return resstock

In [ ]:
def resstock_downsample_to_hour(resstock):
    df = resstock.copy()
    cols_to_drop = [
        'in.county', 'in.geometry_building_type_recs',
        'models_used', 'units_represented'
    ]
    df = df.drop(cols_to_drop, axis=1)
    df = df.resample(rule='H').sum()

    resstock_hr = df

    # TEST: check total is same after resample:
    total_original = resstock['out.site_energy.total.energy_consumption'].sum()
    total_resample = resstock_hr['out.site_energy.total.energy_consumption'].sum()
    if abs((total_original - total_resample)/total_original) > 1e-6:
        print('Error!' + f"original: {total_original}; resample: {total_resample}")
        
    return resstock_hr

In [ ]:
# calculate difference between generation and consumption
# pvwatts: units are W, time steps are h--so the units are actually Wh
# website:
# for this set up, January generation: AC Energy (kWh) = 1,058
# This matches closely, but not exactly, the output that I saved

# ======

# resstock: 
# https://resstock.nrel.gov/page/faq
# "All downloaded energy data is in kWh, including all electricity, natural gas, propane, and fuel oil end uses"

### ResStock floor area info:
* Floor area is not currently included in the residential aggregates, but the floor area can be calculated from the metadata.tsv file (example), by adding up the values in the "floor_area_conditioned_ft_2" column after filtering down to the building type and geographic region corresponding to the pre-aggregated file.

### ResStock:
* How can I see the number of buildings, dwelling units, or number of devices associated with an aggregate load profile from the data viewer? While the pre-aggregated files (example) contain a column with the "floor_area_represented" for commercial or "units_represented" for residential, aggregations generated by the web viewer don’t include the "floor_area_represented" or "units_represented" information currently.

In [ ]:
# I don't know yet how many homes are modeled in ResStock for the area I have
# can normalize the data to assume 1000 kWh/mo, or 12000 kWh/y, of electricity
# but electricity is only two-thirds of the energy
# so really roughly, if we electrify everything, assume 1500 kWh/mo, or 18000 kWh/y, of energy (all electricity)

In [ ]:
# print(f"Total energy (MWh): {int(resstock_hr['out.site_energy.total.energy_consumption'].sum()/1e3)}")
# print(f"Total electricity (MWh): {int(resstock_hr['out.electricity.total.energy_consumption'].sum()/1e3)}")
# print(f"Electricity ratio: {resstock_hr['out.electricity.total.energy_consumption'].sum()/resstock_hr['out.site_energy.total.energy_consumption'].sum()}")

In [ ]:
resstock = resstock_read()

# Merge PVWatts & ResStock

In [ ]:
def create_merged(pvwatts, resstock_hr_norm):
    pvwatts_merge = pvwatts.copy()
    pvwatts_merge['AC System Output (kWh)'] = pvwatts_merge[['AC System Output (Wh)']] / 1e3
    pvwatts_merge = pvwatts_merge.rename(columns={'AC System Output (kWh)': 'PV generation AC (kWh)'})

    merged = pd.concat([
        resstock_hr_norm,
        pvwatts_merge[['PV generation AC (kWh)']],
    ], axis=1)

    # fill in missing value for PV in 2019-01-01 00:00:00 -- nighttime, so no generation
    pvwatts_nan_len = len(merged[merged['PV generation AC (kWh)'].isna()])
    if pvwatts_nan_len==1:
        merged['PV generation AC (kWh)'] = merged['PV generation AC (kWh)'].fillna(0)
    else:
        print("Error!" + f" There was only expected to be 1 nan row, but there were {pvwatts_nan_len} rows")
        
    return merged

In [ ]:
def calculate_shortfall_excess(merged):
    diff = merged['PV generation AC (kWh)'] - merged['Total energy consumption (kWh)']

    shortfall_pv = -1 * diff[diff <= 0]
    shortfall_pv.name = 'PV shortfall kWh'

    excess_pv = diff[diff > 0]
    excess_pv.name = 'PV excess kWh'

    merged = pd.concat([
        merged,
        shortfall_pv,
        excess_pv,
    ], axis=1).fillna(0)

    merged = merged.reset_index()
    
    return merged

In [ ]:
def battery_charge_discharge(merged, battery_capacity_kwh):
    # model battery storage
    # set a certain capacity of battery
    # then step through the time series, to fill the battery when there is excess (up to its limit)
    # and to draw down the battery (only to zero)

    merged['battery kWh'] = float(0) # initialize

    for row in merged.index:
        
        if row == 0:
            battery = float(0)
        else:
            # get value from previous row
            battery = merged.at[row-1, 'battery kWh']

        battery_capacity_kwh = float(battery_capacity_kwh)
        battery_spare_cap = battery_capacity_kwh - battery

        pv_excess = merged.at[row, 'PV excess kWh']
        pv_shortfall = merged.at[row, 'PV shortfall kWh']

        # charge the battery
        # print(f"pv_excess: {pv_excess} & battery_spare_cap: {battery_spare_cap}")
        charged = min(pv_excess, battery_spare_cap)
        merged.at[row, 'charged kWh'] = charged

        battery += charged

        # sold to grid
        sold = pv_excess - charged
        

        # discharge battery
        discharged = min(pv_shortfall, battery)

        battery += -1 * discharged
        merged.at[row, 'discharged kWh'] = discharged
        
        # total shortfall
        total_consump = merged.at[row, 'Total energy consumption (kWh)']
        pv_gen = merged.at[row, 'PV generation AC (kWh)']
        total_supply = pv_gen + discharged
        
        total_shortfall = max(total_consump - total_supply, 0)

        # write values for battery at end of time period
        merged.at[row, 'battery spare kWh'] = battery_spare_cap
        merged.at[row, 'battery kWh'] = battery
        
        merged.at[row, 'bought kWh'] = total_shortfall       
        merged.at[row, 'sold kWh'] = sold

    return merged

# Outputs

In [ ]:
# # batteries are often expressed in terms of number of hours of supply they give you
# kWh_per_hour_avg = merged['Total energy consumption (kWh)'].sum() / (365 * 24)
# hours_of_battery = 8
# print(f"8-hour battery would then be: {round(kWh_per_hour_avg * hours_of_battery, 1)} kWh")

In [ ]:
battery_capacity_kwh = 200
pv_kwh = 200

energy_consumption_per_house = 18000 # kWh

In [ ]:
resstock_hr = resstock_downsample_to_hour(resstock)

number_of_houses_guesstimate = int(resstock['out.site_energy.total.energy_consumption'].sum() / energy_consumption_per_house)
# print(f"Guesstimate for number of houses in ResStock area: {number_of_houses_guesstimate}")

resstock_hr_norm = resstock_hr / number_of_houses_guesstimate

resstock_hr_norm = resstock_hr_norm[['out.site_energy.total.energy_consumption']].rename(columns={
    'out.site_energy.total.energy_consumption': 'Total energy consumption (kWh)',
})

annual_output_MWh = pvwatts['AC System Output (Wh)'].sum()/1e6
round(annual_output_MWh, 1)

pvwatts_scaled = scale_pvwatts(pvwatts, pv_kwh)
merged = create_merged(pvwatts_scaled, resstock_hr_norm)
merged = calculate_shortfall_excess(merged)
merged = battery_charge_discharge(merged, battery_capacity_kwh)
merged = merged.set_index('index')
merged['net bought kWh'] = merged['bought kWh'] - merged['sold kWh']

In [ ]:
merged_sums = merged[['Total energy consumption (kWh)', 'PV generation AC (kWh)',
                      'PV shortfall kWh', 'PV excess kWh', 
                      'charged kWh', 'discharged kWh', 
                      'bought kWh', 'sold kWh', 'net bought kWh']].sum()
merged_sums

In [ ]:
# now = datetime.now() # current date and time
# save_timestamp = now.strftime("%Y-%m-%d %H%M%S")
# merged.to_excel(f'PV home system sim {save_timestamp}.xlsx')

In [ ]:
# average capacity of batteries
merged['battery kWh'].mean()

In [ ]:
# share of time that batteries are at max capacity
len(merged[merged['battery kWh']==battery_capacity_kwh])/len(merged)

In [ ]:
# TO DO: add test to check that total charge = total discharge 
# (can allow some minor discrepancy, to account for any charge remaining at the end of the time period)

In [ ]:
# df = merged.copy()

# df['Hour'] = df.index.hour
# df['Hour'] = df['Hour'].astype(str)

# for row in df.index:
#     hour = df.at[row, 'Hour']
#     if int(hour) >= 8 and int(hour) < 20:
#         df.at[row, 'Hour'] = 'day'
#     else:
#         df.at[row, 'Hour'] = 'night'
        
# # df['Hour'].value_counts()
# df = df.rename(columns={'Hour': 'day-night'})
    
# df['Month'] = df.index.month
# df.groupby(['Month', 'day-night']).sum().astype(int)

In [ ]:
net_supply = merged_sums.at['PV generation AC (kWh)'] + merged_sums.at['bought kWh'] - merged_sums.at['sold kWh']
total_consump = merged_sums.at['Total energy consumption (kWh)'] 

total_consump - net_supply

In [ ]:
# Something seems off, because 

In [ ]:
# Q: how to count the number of cycles?
# Or is there a better way of evaluating, it to look at total charge / total discharge